In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.models import save_model

In [ ]:
# Cifar10データロード
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
# 正規化
x_train = x_train/255.0
x_test = x_test/255.0

In [ ]:
# CNN構築
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(
        24, 3, activation='relu', padding='same',
        input_shape=(32, 32, 3)),  # (32, 32, 24)
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),  # (30, 30, 32)
    tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'), # (15, 15, 32)
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),  # (15, 15, 64)
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),  # (13, 13, 64)
    tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'), # (6, 6, 64)
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Activation('softmax')
])

In [ ]:
# モデル可視化
model.summary()

In [ ]:
# モデルチェックポイントのコールバック
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    '02_cnn_cifar10.hdf5', verbose=1, save_weights_only=False)

In [ ]:
# モデルコンパイル
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# 訓練
model.fit(
    x_train,
    y_train,
    epochs=15,
    validation_data=(x_test, y_test),
    callbacks=[cp_callback]
)

In [ ]:
# モデル評価
val_loss, val_acc = model.evaluate(x_test, y_test, batch_size=128)

In [ ]:
# 保存したモデルのロード
load_model = tf.keras.models.load_model("02_cnn_cifar10.hdf5")

In [ ]:
# テスト画像を1枚ロード
from IPython.display import Image, display_png
from PIL import Image

img = Image.open('image/automobile10.png')
img = img.resize((32, 32))
display_png(img)

In [ ]:
# 入力画像成形、および正規化
x = np.asarray(img)
x = x.reshape(-1, 32, 32, 3)
x = x.astype('float32')
x /= 255

In [ ]:
# 推論実行
predict_result = load_model.predict(x)

In [ ]:
print(predict_result)
print(np.squeeze(predict_result)) # 不要な次元を削除
print(np.argmax(np.squeeze(predict_result)))